# Resolution and Image Quality in OCT

In this notebook, we'll explore what determines image quality in OCT and how to optimize system performance.

## Learning Objectives

By the end of this notebook, you will understand:
1. Axial resolution and its determinants
2. Lateral resolution and focusing constraints
3. The relationship between resolution and imaging depth
4. Trade-offs in OCT system design
5. Sensitivity and dynamic range

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

## 1. Axial Resolution

Axial resolution (depth resolution) determines how well we can distinguish structures along the beam direction.

### Formula

$$\Delta z = \frac{2 \ln 2}{\pi} \frac{\lambda_0^2}{\Delta\lambda} = \frac{2 \ln 2}{\pi} \frac{\lambda_0^2}{n \Delta\lambda}$$

In tissue (with refractive index n ≈ 1.38):

$$\Delta z_{tissue} = \frac{\Delta z_{air}}{n}$$

### Key points:
- **Independent of focusing optics**
- **Determined by source bandwidth**
- Broader bandwidth → Better resolution
- Typical values: 1-15 μm

In [ ]:
def axial_resolution(wavelength_nm, bandwidth_nm, refractive_index=1.0):
    """
    Calculate axial resolution.
    
    Parameters:
    - wavelength_nm: Center wavelength (nm)
    - bandwidth_nm: Source bandwidth (nm)
    - refractive_index: Medium refractive index (default 1.0 for air)
    
    Returns:
    - Axial resolution in micrometers
    """
    wavelength_um = wavelength_nm / 1000
    bandwidth_um = bandwidth_nm / 1000
    
    # Coherence length
    lc = (2 * np.log(2) / np.pi) * (wavelength_um**2 / bandwidth_um)
    
    # Axial resolution (approximately half the coherence length)
    delta_z = lc / 2 / refractive_index
    
    return delta_z

# Common OCT wavelengths and their performance
wavelengths = [840, 1050, 1310]  # nm
bandwidths_range = np.linspace(20, 150, 100)  # nm

plt.figure(figsize=(14, 6))

# Plot for air
plt.subplot(1, 2, 1)
for wl in wavelengths:
    resolutions_air = [axial_resolution(wl, bw, 1.0) for bw in bandwidths_range]
    plt.plot(bandwidths_range, resolutions_air, linewidth=2, label=f'{wl} nm')

plt.xlabel('Bandwidth (nm)', fontsize=12)
plt.ylabel('Axial Resolution (μm)', fontsize=12)
plt.title('Axial Resolution vs. Bandwidth (Air)', fontsize=13, fontweight='bold')
plt.legend(title='Wavelength')
plt.grid(True, alpha=0.3)
plt.ylim(0, 25)

# Plot for tissue
plt.subplot(1, 2, 2)
n_tissue = 1.38
for wl in wavelengths:
    resolutions_tissue = [axial_resolution(wl, bw, n_tissue) for bw in bandwidths_range]
    plt.plot(bandwidths_range, resolutions_tissue, linewidth=2, label=f'{wl} nm')

plt.xlabel('Bandwidth (nm)', fontsize=12)
plt.ylabel('Axial Resolution (μm)', fontsize=12)
plt.title('Axial Resolution vs. Bandwidth (Tissue, n=1.38)', fontsize=13, fontweight='bold')
plt.legend(title='Wavelength')
plt.grid(True, alpha=0.3)
plt.ylim(0, 18)

plt.tight_layout()
plt.show()

# Example calculation
print("Example: 840 nm source with 50 nm bandwidth")
print(f"  Axial resolution in air: {axial_resolution(840, 50, 1.0):.2f} μm")
print(f"  Axial resolution in tissue: {axial_resolution(840, 50, 1.38):.2f} μm")
print("\nKey insight: Broader bandwidth always improves axial resolution!")

## 2. Lateral Resolution

Lateral resolution (transverse resolution) determines how well we can distinguish features perpendicular to the beam.

### Formula

$$\Delta x = \frac{4\lambda f}{\pi D} = \frac{2\lambda}{\pi \text{NA}}$$

where:
- $\lambda$ = wavelength
- $f$ = focal length of the objective
- $D$ = beam diameter
- NA = numerical aperture = $D/(2f)$

### Key points:
- **Determined by focusing optics**
- Better lateral resolution → Shorter depth of focus
- Trade-off between resolution and imaging depth
- Typical values: 10-30 μm

In [ ]:
def lateral_resolution(wavelength_nm, focal_length_mm, beam_diameter_mm):
    """
    Calculate lateral resolution.
    
    Parameters:
    - wavelength_nm: Wavelength (nm)
    - focal_length_mm: Focal length (mm)
    - beam_diameter_mm: Beam diameter at focusing lens (mm)
    
    Returns:
    - Lateral resolution in micrometers
    """
    wavelength_um = wavelength_nm / 1000
    
    delta_x = (4 * wavelength_um * focal_length_mm) / (np.pi * beam_diameter_mm)
    
    return delta_x

def depth_of_focus(wavelength_nm, focal_length_mm, beam_diameter_mm):
    """
    Calculate depth of focus (Rayleigh range).
    
    DOF = π * (Δx)^2 / (4 * λ)
    """
    wavelength_um = wavelength_nm / 1000
    delta_x = lateral_resolution(wavelength_nm, focal_length_mm, beam_diameter_mm)
    
    dof = np.pi * delta_x**2 / (4 * wavelength_um)
    
    return dof

# Explore the trade-off
wavelength = 840  # nm
focal_length = 20  # mm
beam_diameters = np.linspace(0.5, 5, 100)  # mm

lat_res = [lateral_resolution(wavelength, focal_length, d) for d in beam_diameters]
dof = [depth_of_focus(wavelength, focal_length, d) for d in beam_diameters]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Lateral resolution vs beam diameter
ax1.plot(beam_diameters, lat_res, 'b', linewidth=2)
ax1.set_xlabel('Beam Diameter (mm)', fontsize=12)
ax1.set_ylabel('Lateral Resolution (μm)', fontsize=12)
ax1.set_title('Lateral Resolution vs. Beam Diameter', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()  # Smaller is better

# Depth of focus vs lateral resolution
ax2.plot(lat_res, dof, 'r', linewidth=2)
ax2.set_xlabel('Lateral Resolution (μm)', fontsize=12)
ax2.set_ylabel('Depth of Focus (μm)', fontsize=12)
ax2.set_title('The Resolution-DOF Trade-off', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.annotate('Better resolution\n→ Shallower focus', 
             xy=(lat_res[10], dof[10]), xytext=(lat_res[10]+5, dof[10]+200),
             arrowprops=dict(arrowstyle='->', color='black', lw=2),
             fontsize=11, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.show()

print("The Fundamental Trade-off:")
print("  ↑ Lateral resolution → ↓ Depth of focus")
print("  ↓ Lateral resolution → ↑ Depth of focus")
print("\nDesign choice depends on application!")

## 3. Visualizing the Point Spread Function (PSF)

The Point Spread Function describes how a point reflector appears in the OCT image. It's determined by both axial and lateral resolution.

In [ ]:
def oct_psf_2d(axial_res, lateral_res, size=100):
    """
    Generate a 2D OCT point spread function.
    
    Parameters:
    - axial_res: Axial resolution (μm)
    - lateral_res: Lateral resolution (μm)
    - size: Grid size
    """
    # Create coordinate grids
    z = np.linspace(-20, 20, size)  # Axial (depth)
    x = np.linspace(-20, 20, size)  # Lateral
    X, Z = np.meshgrid(x, z)
    
    # Gaussian PSF (simplified model)
    PSF = np.exp(-(X**2 / (2 * (lateral_res/2.355)**2) + 
                   Z**2 / (2 * (axial_res/2.355)**2)))
    
    return X, Z, PSF

# Compare different system configurations
configs = [
    (3, 15, 'High Axial Res\n(Small beam, narrow band)'),
    (8, 8, 'Balanced\n(Medium beam, medium band)'),
    (6, 25, 'High Lateral Res\n(Large beam, medium band)')
]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, (ax_res, lat_res, title) in zip(axes, configs):
    X, Z, PSF = oct_psf_2d(ax_res, lat_res)
    
    im = ax.contourf(X, Z, PSF, levels=20, cmap='hot')
    ax.contour(X, Z, PSF, levels=[0.5], colors='cyan', linewidths=2)
    ax.set_xlabel('Lateral (μm)', fontsize=11)
    ax.set_ylabel('Axial (μm)', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3, color='white', linestyle='--')
    
    # Add text annotations
    ax.text(0.05, 0.95, f'Δz = {ax_res} μm\nΔx = {lat_res} μm',
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontsize=10)

plt.tight_layout()
plt.show()

print("The PSF shape determines what small objects look like in the image.")
print("Cyan contour: Half-maximum intensity (FWHM definition of resolution)")

## 4. Imaging a Test Target

Let's simulate how different resolutions affect imaging of a structured sample.

In [ ]:
def create_test_target(size=200):
    """
    Create a test target with various features.
    """
    target = np.zeros((size, size))
    
    # Vertical lines (test lateral resolution)
    spacings = [20, 10, 5, 3]
    x_start = 20
    for spacing in spacings:
        for i in range(3):
            x = x_start + i * spacing
            if x < size:
                target[20:80, x] = 1.0
        x_start += 60
    
    # Horizontal lines (test axial resolution)
    y_start = 100
    for spacing in spacings:
        for i in range(3):
            y = y_start + i * spacing
            if y < size:
                target[y, 20:180] = 1.0
        y_start += 0  # Stack them
        
    # Small dots (test overall resolution)
    dot_positions = [(150, 50), (150, 80), (150, 110), (150, 140)]
    for y, x in dot_positions:
        if y < size and x < size:
            target[y-1:y+2, x-1:x+2] = 1.0
    
    return target

def simulate_oct_image(target, axial_res, lateral_res):
    """
    Simulate OCT imaging with given resolution.
    """
    # Apply Gaussian blur to simulate resolution limits
    # sigma ≈ resolution / 2.355 (FWHM to sigma conversion)
    sigma_z = axial_res / 2.355  # Axial (vertical)
    sigma_x = lateral_res / 2.355  # Lateral (horizontal)
    
    blurred = gaussian_filter(target, sigma=[sigma_z, sigma_x])
    
    return blurred

# Create target
target = create_test_target()

# Simulate with different resolutions
resolutions = [
    (2, 5, 'Excellent\n(Δz=2, Δx=5 μm)'),
    (5, 10, 'Good\n(Δz=5, Δx=10 μm)'),
    (10, 20, 'Poor\n(Δz=10, Δx=20 μm)')
]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Original target
axes[0].imshow(target, cmap='gray', aspect='equal')
axes[0].set_title('Ideal Target\n(Perfect Resolution)', fontsize=11, fontweight='bold')
axes[0].set_xlabel('Lateral (μm)')
axes[0].set_ylabel('Axial (μm)')
axes[0].axis('off')

# Simulated images
for ax, (ax_res, lat_res, title) in zip(axes[1:], resolutions):
    simulated = simulate_oct_image(target, ax_res, lat_res)
    ax.imshow(simulated, cmap='gray', aspect='equal')
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Notice how fine structures blur together with poorer resolution.")
print("This demonstrates why high resolution is crucial for medical imaging!")

## 5. Sensitivity and Dynamic Range

Beyond resolution, **sensitivity** determines the weakest signal we can detect.

### Sensitivity

Defined as the minimum detectable reflectivity:

$$\text{Sensitivity (dB)} = 10 \log_{10}\left(\frac{P_{reference}}{\text{NEP}}\right)$$

where:
- $P_{reference}$ = reference arm power
- NEP = noise equivalent power

Typical OCT sensitivity: **90-110 dB**

### Dynamic Range

The ratio between the strongest and weakest detectable signals, typically **40-50 dB** in OCT.

In [ ]:
# Simulate sensitivity effects
def add_noise(signal, snr_db):
    """
    Add noise to simulate limited sensitivity.
    
    Parameters:
    - signal: Clean signal
    - snr_db: Signal-to-noise ratio in dB
    """
    signal_power = np.mean(signal**2)
    noise_power = signal_power / (10**(snr_db/10))
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

# Create a multi-layer structure with varying reflectivity
depth = np.linspace(0, 300, 1000)
signal = np.zeros_like(depth)

# Add layers with different reflectivities
layers = [
    (50, 1.0, 'Strong (0 dB)'),
    (100, 0.5, 'Medium (-6 dB)'),
    (150, 0.1, 'Weak (-20 dB)'),
    (200, 0.03, 'Very weak (-30 dB)'),
    (250, 0.01, 'Extremely weak (-40 dB)')
]

for pos, refl, name in layers:
    signal += refl * np.exp(-((depth - pos) / 5)**2)

# Simulate with different sensitivities
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
snr_values = [40, 30, 20, 10]  # dB

for ax, snr in zip(axes.flat, snr_values):
    noisy_signal = add_noise(signal, snr)
    
    ax.plot(depth, noisy_signal, 'b', linewidth=1, alpha=0.7)
    ax.plot(depth, signal, 'r--', linewidth=2, alpha=0.5, label='True signal')
    
    for pos, refl, name in layers:
        ax.axvline(pos, color='g', linestyle=':', alpha=0.5)
    
    ax.set_xlabel('Depth (μm)', fontsize=11)
    ax.set_ylabel('Signal (a.u.)', fontsize=11)
    ax.set_title(f'SNR = {snr} dB (Sensitivity ≈ {100-snr} dB)', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.5, 1.5)

plt.tight_layout()
plt.show()

print("As sensitivity decreases (lower SNR):")
print("  - Weak reflectors become harder to detect")
print("  - Noise floor increases")
print("  - Imaging depth decreases")
print("\nHigh sensitivity is crucial for deep tissue imaging!")

## 6. System Design Trade-offs

Let's summarize the key trade-offs in OCT system design:

In [ ]:
# Create a trade-off comparison table
import pandas as pd

# Define three system configurations
systems = {
    'Parameter': ['Wavelength (nm)', 'Bandwidth (nm)', 'Beam Diameter (mm)', 
                  'Focal Length (mm)', 'Axial Res (μm)', 'Lateral Res (μm)', 
                  'DOF (μm)', 'Best For'],
    'High Axial Res': [840, 100, 1.5, 20, 2.5, 18, 350, 'Detailed depth\nstructure'],
    'Balanced': [1050, 70, 2.5, 20, 5.5, 11, 130, 'General purpose\nimaging'],
    'High Lateral Res': [840, 50, 4.0, 20, 6.0, 7, 65, 'Fine lateral\ndetails']
}

df = pd.DataFrame(systems)
print("="*70)
print("OCT System Design Comparison")
print("="*70)
print(df.to_string(index=False))
print("="*70)

# Visualize the trade-offs
fig, ax = plt.subplots(figsize=(10, 8))

# Extract numeric data for plotting
ax_res = [2.5, 5.5, 6.0]
lat_res = [18, 11, 7]
dof = [350, 130, 65]
names = ['High Axial Res', 'Balanced', 'High Lateral Res']
colors = ['red', 'blue', 'green']

# Create scatter plot
for i, (name, color) in enumerate(zip(names, colors)):
    size = dof[i]  # Size represents depth of focus
    ax.scatter(lat_res[i], ax_res[i], s=size*3, alpha=0.6, 
              color=color, label=name, edgecolors='black', linewidth=2)
    ax.annotate(name, (lat_res[i], ax_res[i]), 
               xytext=(10, 10), textcoords='offset points',
               fontsize=11, fontweight='bold',
               bbox=dict(boxstyle='round', facecolor=color, alpha=0.3))

ax.set_xlabel('Lateral Resolution (μm)', fontsize=13, fontweight='bold')
ax.set_ylabel('Axial Resolution (μm)', fontsize=13, fontweight='bold')
ax.set_title('OCT System Design Space\n(Bubble size ∝ Depth of Focus)', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.invert_xaxis()  # Better resolution to the right
ax.invert_yaxis()  # Better resolution to the top

# Add annotations for "better" directions
ax.text(0.95, 0.05, 'Better lateral →', transform=ax.transAxes,
       ha='right', fontsize=10, style='italic')
ax.text(0.05, 0.95, '↑\nBetter\naxial', transform=ax.transAxes,
       ha='left', va='top', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("1. Cannot optimize all parameters simultaneously")
print("2. System design depends on application requirements")
print("3. Larger bubbles = larger depth of focus (good for thick samples)")
print("4. Smaller bubbles = better lateral resolution (but limited DOF)")

## Summary

In this notebook, we've learned:

1. ✅ **Axial resolution**: Determined by source bandwidth, independent of optics

2. ✅ **Lateral resolution**: Determined by focusing optics, trades off with depth of focus

3. ✅ **Point Spread Function**: Describes how point objects appear in images

4. ✅ **Sensitivity**: Determines minimum detectable signal, crucial for imaging depth

5. ✅ **Design trade-offs**: Cannot optimize all parameters simultaneously

## Key Formulas

**Axial Resolution:**
$$\Delta z = \frac{2 \ln 2}{\pi} \frac{\lambda_0^2}{n \Delta\lambda}$$

**Lateral Resolution:**
$$\Delta x = \frac{4\lambda f}{\pi D}$$

**Depth of Focus:**
$$\text{DOF} = \frac{\pi (\Delta x)^2}{4\lambda}$$

## Design Guidelines

1. **For better axial resolution**: Increase bandwidth
2. **For better lateral resolution**: Increase beam diameter (but decreases DOF)
3. **For larger imaging depth**: Optimize sensitivity, accept lower lateral resolution
4. **For specific applications**: Choose wavelength based on tissue properties

## Next Steps

In the next notebook, we'll explore:
- Time-Domain OCT implementation
- How mechanical scanning generates A-scans
- Building a complete TD-OCT simulation

---

**Continue to Notebook 4: Time-Domain OCT →**